In [121]:
import pandas as pd
import numpy as np

In [122]:
def generate_metadata(dataframe):
    """
    Generates a DataFrame containing metadata for the columns of the provided DataFrame.

    :param dataframe: DataFrame for which metadata will be generated.
    :return: DataFrame containing metadata.
    """

    # Collection of basic metadata
    metadata = pd.DataFrame({
        'variable': dataframe.columns,
        'type': dataframe.dtypes,
        'null_count': dataframe.isnull().sum(),
        'null_percent': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinality': dataframe.nunique(),
    })
#     metadata = metadata.sort_values(by='type')
    metadata = metadata.reset_index(drop=True)

    return metadata

In [123]:
train_path = "./train.csv"
test_path = "./test.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [124]:
metadata = generate_metadata(train_df)
print(metadata)

       variable     type  null_count  null_percent  cardinality
0   PassengerId    int64           0          0.00          891
1      Survived    int64           0          0.00            2
2        Pclass    int64           0          0.00            3
3          Name   object           0          0.00          891
4           Sex   object           0          0.00            2
5           Age  float64         177         19.87           88
6         SibSp    int64           0          0.00            7
7         Parch    int64           0          0.00            7
8        Ticket   object           0          0.00          681
9          Fare  float64           0          0.00          248
10        Cabin   object         687         77.10          147
11     Embarked   object           2          0.22            3


In [125]:
## Processing the cabin from C123, D32, ...  -> 3.0, 4.0, ... (take the first char, then turn it into float)
# convert the cabin into numerical value
def to_numerical(char):
    # use if incase of nan value
    if pd.notna(char):
        return ord(char)-ord('A')+1
    return np.nan

# take the first char in cabin, then change it into numerical using to_numerical function
def fix_cabin(df):
    df["Cabin"] = df["Cabin"].str[0]
    df["Cabin"] = df["Cabin"].apply(lambda x: to_numerical(x))
    return df

In [126]:
# use train dataset to find filler
# filler is used to fill the nan value (filler is based on the pclass)
def find_filler(df):
    # find the average age per class
    average_age_class = df.groupby('Pclass')['Age'].mean()
    # find the mode embarked per class
    mode_embarked_class = df.groupby('Pclass')['Embarked'].agg(lambda x: x.mode().iloc[0])
    # find the mode of cabin
    mode_cabin_class = df.groupby('Pclass')['Cabin'].agg(lambda x: x.mode().iloc[0])

    average_age_class = list(average_age_class)
    mode_embarked_class = list(mode_embarked_class)
    mode_cabin_class = list(mode_cabin_class)

    return average_age_class+mode_cabin_class+mode_embarked_class

# nan value is filled based on the Pclass, hence why the long repetitive code
# fill nan value of age, cabin, embarked (connected to fill_na function)
def fill(df, age, cabin, embark):
  replace = {"Age":age, "Cabin":cabin, "Embarked":embark}
  for col, value in replace.items():
    df.loc[df[col].isna(), col] = value
  return df

# fill the nan value (connected to preprocessing function)
## fill na -> split the dataset into 3 based on the class, then fill nan of each class, lastly combine the dataset (if train dataset = shuffle, if test dataset = sort based on id)
def fill_na(df, prep_type):

    # split into 3 based on class
    df1 = df.loc[df["Pclass"]==1]
    df2 = df.loc[df["Pclass"]==2]
    df3 = df.loc[df["Pclass"]==3]
    df_list = [df1, df2, df3]
    
    # shape probably like this [age1, age2, age3, mode1, mode2, mode3, ...]
    filler = find_filler(train_df)
    
    print(filler)
    # fill the nan value in df1, df2, df33
    for idx, x in enumerate(df_list):
        # what is id, 3+idx, 6+idx?
        # basically the filler shape is (age(class1), age(class2), age(class3), cabin(class1), ..., embarked(class3))
        df_list[idx] = fill(x, int(filler[idx]), filler[3+idx], filler[6+idx])

    # concat all the separated df
    final_df = pd.concat([df_list[0], df_list[1], df_list[2]])
    
    # if train df, shuffle the dataset
    if prep_type == "Train":
        final_df = final_df.sample(frac=1, random_state=64)
    # if test df, sort the dataset based on passenger id (to match the correct answer)
    else:
        final_df.sort_values(by=["PassengerId"], inplace=True)
    
    return final_df

In [127]:
## this is basically combining the fix cabin, fill na, and dropping some column (also changing sex into numerical)
def preprocess(df, type):
    
    # in test we dont drop the passenger id to sort the answer
    if type == "Train":
        # drop passenger id
        df.drop("PassengerId", axis=1, inplace=True)

    # dropping name, fare, ticket
    df.drop(columns=["Name", "Fare", "Ticket"], inplace=True)
    
    # changing sex  into numerical
    df["Sex"].replace("male", 1, inplace=True)
    df["Sex"].replace("female", 0, inplace=True)

    # fix the cabin
    df = fix_cabin(df)

    # input fillna here
    df = fill_na(df, type)

    # change embarked into numerical
    df["Embarked"].replace('C', 1, inplace=True)
    df["Embarked"].replace('Q', 2, inplace=True)
    df["Embarked"].replace('S', 3, inplace=True)

    # split x and y
    # case: 
    # Train: x -> feature, y -> survived
    # Test: x -> feature, y -> PassengerID
    labels = list(df.columns)
    x = df[labels[1:]]
    y = df[labels[0]]

    return x, y

In [128]:
x_train, y_train = preprocess(train_df, "Train")
x_test, y_test = preprocess(test_df, "Test")

[38.233440860215055, 29.87763005780347, 25.14061971830986, 3.0, 6.0, 6.0, 'S', 'S', 'S']


,Pclass,Sex,Age,SibSp,Parch,Cabin,Embarked
779,1,0,43.0,0,1,2.0,3
593,3,0,25.0,0,2,6.0,2
17,2,1,29.0,0,0,6.0,3
346,2,0,40.0,0,0,6.0,3
767,3,0,30.5,0,0,6.0,2


In [129]:
y_train.head()

779    1
593    0
17     1
346    1
767    0
Name: Survived, dtype: int64

In [130]:
metadata = generate_metadata(x_train)
print(metadata)

   variable     type  null_count  null_percent  cardinality
0    Pclass    int64           0           0.0            3
1       Sex    int64           0           0.0            2
2       Age  float64           0           0.0           88
3     SibSp    int64           0           0.0            7
4     Parch    int64           0           0.0            7
5     Cabin  float64           0           0.0            8
6  Embarked    int64           0           0.0            3
